In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv('data/casas.csv')

In [3]:
df.head()

,tamanho,ano,garagem,preco
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [4]:
X = df.drop('preco',axis=1)
y = df['preco'].copy()

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [7]:
y_train.shape, y_test.shape

((1022,), (438,))

In [8]:
import mlflow
import dagshub

dagshub.init("MLOPS-challenge", "sarahgruetz", mlflow=True)

Token Dagshub OAuth token, valid until 2023-07-14 12:09:04.280461+00:00 does not exist in the storage
Token Dagshub OAuth token, valid until 2023-07-25 12:32:50.304156+00:00 does not exist in the storage
Token Dagshub OAuth token, valid until 2023-08-03 13:44:27.174600+00:00 does not exist in the storage
Token Dagshub OAuth token, valid until 2023-08-11 13:13:06.176531+00:00 does not exist in the storage
Token Dagshub OAuth token, valid until 2023-08-16 17:22:57.304244+00:00 does not exist in the storage
Token Dagshub OAuth token, valid until 2023-08-25 19:58:07.999609+00:00 does not exist in the storage


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\sarah\anaconda3\envs\mlops-course\Lib\site-packages\rich\live.py:229: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=923585f8-acef-42be-bfeb-bfab47de5935&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1f4e1cadec86580d6167170296d668590e8e6a175fe0bb2a225ac6253ee4ea76




Initialized MLflow to track repo "sarahgruetz/MLOPS-challenge"

Repository sarahgruetz/MLOPS-challenge initialized!

In [ ]:
#mlflow.set_experiment('house-price-predictions')

2024/04/26 16:01:54 INFO mlflow.tracking.fluent: Experiment with name 'house-price-predictions' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/sarah/OneDrive/Documentos/MLOps/MLFlow/mlflow/notebooks/mlruns/216420245913578639', creation_time=1714158114908, experiment_id='216420245913578639', last_update_time=1714158114908, lifecycle_stage='active', name='house-price-predictions', tags={}>

# Linear Regression

In [9]:
mlflow.start_run()

<ActiveRun: >

In [10]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [11]:
mlflow.sklearn.log_model(lr,'lr')

In [12]:
lr.predict(X_test)[:2]

array([119279.7701544 , 289022.71460536])

In [13]:
lr_predict = lr.predict(X_test)

In [14]:
from sklearn.metrics import r2_score, root_mean_squared_error

In [15]:
rmse = root_mean_squared_error(y_test, lr_predict)
r2 = r2_score(y_test, lr_predict)
mlflow.log_metric('rmse',rmse)
mlflow.log_metric('r2',r2)

In [16]:
print(rmse, r2)

45592.399782518485 0.7021153642898048


In [17]:
mlflow.end_run()

# XGBoost

In [18]:
from xgboost import XGBRFRegressor

In [32]:
xgb_params = {
    'learning_rate':1,
    #'n_estimators':20,
    'subsample':0.8,
    #'max_depth':5,
    'random_state':42
}

with mlflow.start_run():
    xgb = XGBRFRegressor(**xgb_params)
    xgb.fit(X_train, y_train)
    mlflow.xgboost.autolog()

    xgb_predict = xgb.predict(X_test)

    rmse = root_mean_squared_error(y_test, xgb_predict)
    r2 = r2_score(y_test, xgb_predict)

    mlflow.log_params(xgb_params)
    mlflow.log_metric('rmse',rmse)
    mlflow.log_metric('r2',r2)

2024/05/15 17:49:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\sarah\anaconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2024/05/15 17:49:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\sarah\anaconda3\envs\mlops-c

In [24]:
print(rmse, r2)

36521.41859916094 0.8088569654994723


In [33]:
import pickle

filename = 'pricing_model.pkl'
pickle.dump(xgb, open(filename, 'wb'))